This notebook formats 2008 and 2014 city council election results with the 7 main parties. We can then use these data to model election results at the district level ("arrondissement"). We'll do that in another notebook.

In [1]:
%load_ext lab_black

import numpy as np
import pandas as pd

from typing import Set

AFFILIATIONS = {
    "farleft": ["LFG", "LEXG"],
    "left": ["LUG", "LSOC"],
    "green": ["LVEC"],
    "right": ["LUMP", "LUDI", "LUD"],
    "farright": ["LFN"],
}

First, here are some helper functions:

In [2]:
def extract_nuances(nuances_df: pd.DataFrame) -> Set[str]:
    """
    Extract the nuances competing in this election.
    From the dataframe of nuances, we check each column for each line. 
    If the cell is not empty and the nuance is not already counted, we add it to the set of nuances.
    """
    nuances_set = set()

    for _, line in nuances_df.iterrows():
        for col in nuances_df.columns:
            if pd.notnull(line[col]):
                nuances_set.update({line[col]})

    return nuances_set


def format_results(df: pd.DataFrame, nuances_set: Set[str]) -> pd.DataFrame:
    """
    Take the raw df, for each line switch the nuance's label to column name, 
    and match it with the corresponding score of this party.
    Return a dataframe with the proper format.
    """
    res = {
        "date": df.date.values,
        "ville": df.ville.values,
        "arrondissement": df.arrondissement.values,
        "Exprimés": df["Exprimés"].values,
    }
    res.update({nuance: [] for nuance in nuances_set})

    nuances_lbls = df.filter(like="Code Nuance").columns
    scores_lbls = df.filter(like="Voix").columns

    # each line is an arrondissement:
    for _, line in df.iterrows():
        tempset = nuances_set.copy()

        # iterate over nuances in line:
        for n, s in zip(nuances_lbls, scores_lbls):
            name = line[n]
            score = line[s]
            if pd.notnull(name):
                # if 1st time we see this nuance in this line:
                if name in tempset:
                    res[name].append(score)
                    tempset.remove(name)
                # if we already saw this nuance in this line:
                else:
                    res[name][-1] += score
        # if nuance still in tempset after iteration, then it's not competing in this arrondissement:
        for nuance in tempset:
            res[nuance].append(np.nan)

    return pd.DataFrame(data=res)


def attribute_parties(df: pd.DataFrame, nuances_set: Set[str]) -> pd.DataFrame:
    """
    From a dataframe with general party denomination, attribute colloquial party names and 
    add parties with same nuance.
    Then aggregate other parties and drop all useless ones.
    """
    for p in AFFILIATIONS.keys():
        # which candidate represents the party this year?
        intersection = list(nuances_set & set(AFFILIATIONS[p]))
        # take only LFI for farleft, starting in 2012:
        if "LFG" in intersection:
            df = df.rename(columns={"LFG": p})
        else:
            # add candidates with same nuance, then drop:
            if len(intersection) >= 2:
                df[p] = df[intersection].sum(axis=1)
                df.drop(intersection, axis=1, inplace=True)
            # rename column of only candidate of this party:
            elif len(intersection) == 1:
                df = df.rename(columns={intersection[0]: p})

    # aggregate other parties:
    rest = df.filter(like="L")
    df["other"] = rest.sum(axis=1)
    df.drop(rest.columns, axis=1, inplace=True)

    return df

### Municipales 2014
Let's begin by formatting 2014 city council election results - as they are already at the district level ("arrondissement") it will be easier.

In [62]:
d = pd.read_excel("data/election_results_1st_round/munic2014-ardmnt.xlsx")
d["date"] = d["Date de l'export"].dt.normalize()  # only interested in the date
d["ville"], _, d["arrondissement"] = d["Libellé de la commune"].str.split().str
d["arrondissement"] = d["arrondissement"].astype(int)
d = d.sort_values(["ville", "arrondissement"])
d.head()

,Date de l'export,Code du département,Type de scrutin,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,...,Liste.10,Sièges / Elu.10,Sièges Secteur.10,Sièges CC.10,Voix.10,% Voix/Ins.10,% Voix/Exp.10,date,ville,arrondissement
0,2014-03-25 12:52:00,69,LI2,RHONE,123SR01,Lyon secteur 1,16482,6936,42.08,9546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,Lyon,1
1,2014-03-25 12:52:00,69,LI2,RHONE,123SR02,Lyon secteur 2,16863,6658,39.48,10205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,Lyon,2
2,2014-03-25 12:52:00,69,LI2,RHONE,123SR03,Lyon secteur 3,52133,22494,43.15,29639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,Lyon,3
3,2014-03-25 12:52:00,69,LI2,RHONE,123SR04,Lyon secteur 4,22557,9096,40.32,13461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,Lyon,4
4,2014-03-25 12:52:00,69,LI2,RHONE,123SR05,Lyon secteur 5,28373,11724,41.32,16649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,Lyon,5


The main difficulty lies in the fact that party labels (what we'll call "nuances") and party results are separated (they each have their dedicated column). Our first task is then to match each label with its results. Let's begin by isolating the useful columns:

In [63]:
subset = ["date", "ville", "arrondissement", "Exprimés"]
for n, s in zip(
    d.filter(like="Code Nuance").columns, d.columns[d.columns.str.startswith("Voix")]
):
    subset.append(n)
    subset.append(s)
d = d[subset]
d.head()

,date,ville,arrondissement,Exprimés,Code Nuance,Voix,Code Nuance.1,Voix.1,Code Nuance.2,Voix.2,...,Code Nuance.6,Voix.6,Code Nuance.7,Voix.7,Code Nuance.8,Voix.8,Code Nuance.9,Voix.9,Code Nuance.10,Voix.10
0,2014-03-25,Lyon,1,9433,LEXG,86,LFG,3156,LSOC,2447,...,LFN,583.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-03-25,Lyon,2,10055,LFG,487,LSOC,2737,LVEC,609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-03-25,Lyon,3,29134,LFG,1579,LSOC,11256,LVEC,2854,...,LFN,3603.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-03-25,Lyon,4,13199,LEXG,123,LFG,1323,LSOC,4522,...,LDIV,375.0,LUD,3493.0,LFN,1131.0,NaN,NaN,NaN,NaN
4,2014-03-25,Lyon,5,16405,LEXG,154,LFG,752,LSOC,5954,...,LFN,1857.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The functions `extract_nuances` and `format_results` are designed to do just that: extract the unique nuances competing in this election, and then match each nuance with the corresponding score of the party:

In [64]:
nuances_set = extract_nuances(d.filter(like="Code Nuance"))
d = format_results(d, nuances_set)

d["LFG"] = d["LFG"].fillna(d["LPG"])  # same party
d = d.drop("LPG", axis=1)
d.head()

,date,ville,arrondissement,Exprimés,LVEC,LFN,LDVD,LUMP,LUDI,LEXG,LDIV,LFG,LUG,LUD,LDVG,LSOC
0,2014-03-25,Lyon,1,9433,1064.0,583.0,NaN,NaN,NaN,86.0,293.0,3156.0,NaN,1804.0,NaN,2447.0
1,2014-03-25,Lyon,2,10055,609.0,1159.0,NaN,NaN,NaN,NaN,325.0,487.0,NaN,4738.0,NaN,2737.0
2,2014-03-25,Lyon,3,29134,2854.0,3603.0,NaN,NaN,NaN,NaN,1681.0,1579.0,NaN,8161.0,NaN,11256.0
3,2014-03-25,Lyon,4,13199,1567.0,1131.0,NaN,NaN,NaN,123.0,630.0,1323.0,NaN,3493.0,410.0,4522.0
4,2014-03-25,Lyon,5,16405,1340.0,1857.0,NaN,NaN,NaN,154.0,498.0,752.0,NaN,5850.0,NaN,5954.0


Now we have to attribute each general party denomination to its colloquial party name. However, some parties may have the same ideological leaning, or they have made alliance, so we have to add them together. The function `attribute_parties` take care of it, and then aggregates the rest of the parties into the category "other". It then drops all useless parties:

In [65]:
d = attribute_parties(d_, nuances_set)
d = d.rename(columns={"Exprimés": "N"})
d = d.reindex(
    ["date", "ville", "arrondissement", "N"] + list(AFFILIATIONS.keys()) + ["other"],
    axis=1,
)
d

,date,ville,arrondissement,N,farleft,left,green,right,farright,other
0,2014-03-25,Lyon,1,9433,3156.0,2447.0,1064.0,1804.0,583.0,379.0
1,2014-03-25,Lyon,2,10055,487.0,2737.0,609.0,4738.0,1159.0,325.0
2,2014-03-25,Lyon,3,29134,1579.0,11256.0,2854.0,8161.0,3603.0,1681.0
3,2014-03-25,Lyon,4,13199,1323.0,4522.0,1567.0,3493.0,1131.0,1163.0
4,2014-03-25,Lyon,5,16405,752.0,5954.0,1340.0,5850.0,1857.0,652.0
5,2014-03-25,Lyon,6,17920,561.0,4801.0,1110.0,8971.0,1867.0,610.0
6,2014-03-25,Lyon,7,19902,1543.0,7724.0,2165.0,4746.0,2597.0,1127.0
7,2014-03-25,Lyon,8,18543,1008.0,7473.0,1435.0,4303.0,3421.0,903.0
8,2014-03-25,Lyon,9,12220,686.0,5581.0,921.0,2701.0,1684.0,647.0
9,2014-03-25,Marseille,1,23480,2108.0,6331.0,NaN,9063.0,3526.0,2452.0


### Municipales 2008

And with that, we're done formatting 2014 city council elections. Let's now turn to the 2008 election results, which present the same structure, but are disaggregated at the ballot box level - so we must first aggregate them at the district level:

In [58]:
df = pd.read_excel("data/election_results_1st_round/munic2008-bdv.xlsx")
df = df.rename(columns={"Libellé de la commune": "ville"})
df = df[df.ville == "Paris"].sort_values(["Code du b.vote"])
df["date"] = df["Date de l'export"].dt.normalize()

# Infer arrondissement from ballot-box number:
df["arrondissement"], _ = (df["Code du b.vote"] / 100).astype(str).str.split(".").str
df["arrondissement"] = df["arrondissement"].astype(int)
df.head()

,Date de l'export,Code du departement,Libelle du departement,Code de la commune,ville,Code du b.vote,Inscrits,Abstentions,% Abs/Ins,Votants,...,Sexe.11,Nom.11,Prenom.11,Liste.11,Sieges.11,Voix.11,% Voix/Ins.11,% Voix/Exp.11,date,arrondissement
0,2008-05-23 16:09:14,75,PARIS,56,Paris,101,1047,397,37.92,650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-23,1
1,2008-05-23 16:09:15,75,PARIS,56,Paris,102,887,352,39.68,535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-23,1
2,2008-05-23 16:09:16,75,PARIS,56,Paris,103,1393,607,43.58,786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-23,1
3,2008-05-23 16:09:16,75,PARIS,56,Paris,104,1285,535,41.63,750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-23,1
4,2008-05-23 16:09:16,75,PARIS,56,Paris,105,1000,378,37.80,622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-23,1


In [59]:
nuances_lbls = df.filter(like="Code Nuance").columns.tolist()
nuances_df = df[["date", "ville", "arrondissement"] + nuances_lbls]
# aggregate nuances by arrondissement (take first one because nuances are the same for all ballot box in each arrondissement):
nuances_df = nuances_df.groupby("arrondissement").first()

scores_lbls = df.columns[df.columns.str.startswith("Voix")].tolist()
scores_df = df[["arrondissement", "Exprimés"] + scores_lbls]
# add all scores for each party, in each arrondissement:
scores_df = scores_df.groupby("arrondissement").sum()

# aggregate nuances and scores by arrondissement:
df = nuances_df.join(scores_df).reset_index()
# reorder columns:
reorder = ["date", "ville", "arrondissement", "Exprimés"]
for n, s in zip(nuances_lbls, scores_lbls):
    reorder.append(n)
    reorder.append(s)
df = df[reorder]
df.head()

,date,ville,arrondissement,Exprimés,Code Nuance,Voix,Code Nuance.1,Voix.1,Code Nuance.2,Voix.2,...,Code Nuance.7,Voix.7,Code Nuance.8,Voix.8,Code Nuance.9,Voix.9,Code Nuance.10,Voix.10,Code Nuance.11,Voix.11
0,2008-05-23,Paris,1,6127,LEXG,75,LSOC,2289,LVEC,439,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,2008-05-23,Paris,2,6736,LEXG,90,LSOC,2231,LVEC,2016,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,2008-05-23,Paris,3,11974,LEXG,92,LEXG,133,LSOC,6685,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,2008-05-23,Paris,4,10573,LEXG,151,LSOC,5127,LVEC,834,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,2008-05-23,Paris,5,23614,LEXG,125,LEXG,563,LSOC,8187,...,LDVD,140.0,LFN,418.0,NaN,0.0,NaN,0.0,NaN,0.0


In [63]:
nuances_set = extract_nuances(df.filter(like="Code Nuance"))
format_results(df, nuances_set)

,date,ville,arrondissement,Exprimés,LDVD,LEXG,LFN,LAUT,LVEC,LSOC,LCMD,LMAJ,LDVG
0,2008-05-23,Paris,1,6127,NaN,75.0,152.0,NaN,439.0,2289.0,531.0,2641.0,NaN
1,2008-05-23,Paris,2,6736,68.0,90.0,167.0,NaN,2016.0,2231.0,621.0,1543.0,NaN
2,2008-05-23,Paris,3,11974,NaN,225.0,258.0,NaN,1237.0,6685.0,1111.0,2458.0,NaN
3,2008-05-23,Paris,4,10573,NaN,151.0,286.0,NaN,834.0,5127.0,863.0,3312.0,NaN
4,2008-05-23,Paris,5,23614,691.0,688.0,418.0,NaN,1287.0,8187.0,3385.0,8958.0,NaN
5,2008-05-23,Paris,6,15488,NaN,NaN,356.0,577.0,590.0,5166.0,1530.0,7269.0,NaN
6,2008-05-23,Paris,7,17967,850.0,NaN,537.0,252.0,535.0,4080.0,2819.0,8894.0,NaN
7,2008-05-23,Paris,8,12325,4376.0,NaN,293.0,109.0,318.0,2302.0,808.0,4119.0,NaN
8,2008-05-23,Paris,9,20643,NaN,602.0,567.0,NaN,1299.0,10163.0,1659.0,6353.0,NaN
9,2008-05-23,Paris,10,28359,1275.0,2158.0,837.0,898.0,2564.0,13766.0,2348.0,4513.0,NaN


In [ ]:
# https://fr.wikipedia.org/wiki/%C3%89lections_municipales_de_2008_%C3%A0_Paris#R%C3%A9sultats_par_arrondissement

d["LFG"] = d["LFG"].fillna(d["LPG"])  # same party
d = d.drop("LPG", axis=1)
d.head()

d = attribute_parties(d_, nuances_set)
d = d.rename(columns={"Exprimés": "N"})
d = d.reindex(
    ["date", "ville", "arrondissement", "N"] + list(AFFILIATIONS.keys()) + ["other"],
    axis=1,
)
d